<img src="https://i.imgur.com/FoKB5Z5.png" align="left" width="300" height="250" title="source: imgur.com" /></a>

## Program Code: J620-002-4:2020 

## Program Name: FRONT-END SOFTWARE DEVELOPMENT

## Title :  Hypothesis Testing with NHANES Dataset

#### Name: Phua Yan Han

#### IC Number: 050824070059

#### Date : 14/7/23

#### Introduction : 



#### Conclusion :






# Case Study - Hypothesis testing with NHANES Data

In this notebook we will explore formal hypothesis testing using the [NHANES](https://www.cdc.gov/nchs/nhanes/index.htm) data.

It is important to note that the NHANES data are a "complex survey".  The data are not an independent and representative sample from the target population.  Proper analysis of complex survey data should make use of additional information about how the data were collected.  Since complex survey analysis is a somewhat specialized topic, we ignore this aspect of the data here, and analyze the NHANES data as if it were an independent and identically distributed sample from a population.

First we import the libraries that we will need.

In [150]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats.distributions as dist

Below we read the data, and convert some of the integer codes to text values.  The NHANES codebooks for
[SMQ020](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/SMQ_I.htm#SMQ020),
[RIAGENDR](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#RIAGENDR), and
[DMDCITZN](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDCITZN) describe the meanings of the numerical
codes.

In [151]:
# read data
da = pd.read_csv("../Data files/nhanes_2015_2016.csv")
da.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


In [152]:
## for 'SMQ020x', replace 1: Yes, 2: No, 7: Nan, 9: Nan
da['SMQ020x']=da['SMQ020']
reset = {1:'Yes',2:'No',7:None,9:None}
da = da.replace({'SMQ020x':reset})
da['SMQ020x']

0       Yes
1       Yes
2       Yes
3        No
4        No
       ... 
5730    Yes
5731     No
5732    Yes
5733    Yes
5734     No
Name: SMQ020x, Length: 5735, dtype: object

In [153]:
## for 'RIAGENDRx' replace 1: Male, and 2: Female
da['RIAGENDRx']=da['RIAGENDR']
reset = {1:'Male',2:'Female'}
da = da.replace({'RIAGENDRx':reset})
da['RIAGENDRx']

0         Male
1         Male
2         Male
3       Female
4       Female
         ...  
5730    Female
5731      Male
5732    Female
5733      Male
5734    Female
Name: RIAGENDRx, Length: 5735, dtype: object

In [154]:
## for 'DMDCITZNx' replace 1: Yes, 2: No, 7: Nan, 9: Nan
da['DMDCITZNx']=da['DMDCITZN']
reset = {1:'Yes',2:'No',7:None,9:None}
da = da.replace({'DMDCITZNx':reset})
da['DMDCITZNx']

0       Yes
1        No
2       Yes
3       Yes
4       Yes
       ... 
5730    Yes
5731    Yes
5732    Yes
5733     No
5734    Yes
Name: DMDCITZNx, Length: 5735, dtype: object

### Hypothesis tests for two proportions

Comparative tests tend to be used much more frequently than tests comparing one population to a fixed value.  A two-sample test of proportions is used to assess whether the proportion of individuals with some trait differs between two sub-populations.  For example, we can compare the smoking rates between females and males. Since smoking rates vary strongly with age, we do this in the subpopulation of people between 20 and 25 years of age.  In the cell below, we carry out this test without using any libraries, implementing all the test procedures covered elsewhere in the course using Python code.  We find that the smoking rate for men is around 10 percentage points greater than the smoking rate for females, and this difference is statistically significant (the p-value is around 0.01).

In [155]:
# Drop missing values from SMQ020x, RIDAGEYR and RIAGENDRx
smoke_da = da.copy()
smoke_da.dropna(subset=['SMQ020x','RIDAGEYR','RIAGENDRx'], inplace = True)
smoke_da[['SMQ020x','RIDAGEYR','RIAGENDRx']]

,SMQ020x,RIDAGEYR,RIAGENDRx
0,Yes,62,Male
1,Yes,53,Male
2,Yes,78,Male
3,No,56,Female
4,No,42,Female
...,...,...,...
5730,Yes,76,Female
5731,No,26,Male
5732,Yes,80,Female
5733,Yes,35,Male


In [156]:
smoke_da

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210,SMQ020x,RIAGENDRx,DMDCITZNx
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,184.5,27.8,43.3,43.6,35.9,101.1,2.0,Yes,Male,Yes
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,171.4,30.8,38.0,40.0,33.2,107.9,NaN,Yes,Male,No
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,170.1,28.8,35.6,37.0,31.0,116.5,2.0,Yes,Male,Yes
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,160.9,42.4,38.5,37.7,38.3,110.1,2.0,No,Female,Yes
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,164.9,20.3,37.4,36.0,27.2,80.4,2.0,No,Female,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5730,93695,2.0,2.0,NaN,1,2,76,3,1.0,3.0,...,165.8,21.5,38.2,37.0,29.5,95.0,2.0,Yes,Female,Yes
5731,93696,2.0,2.0,NaN,2,1,26,3,1.0,5.0,...,182.2,33.8,43.4,41.8,42.3,110.2,2.0,No,Male,Yes
5732,93697,1.0,NaN,1.0,1,2,80,3,1.0,4.0,...,152.2,31.0,31.3,37.5,28.8,NaN,2.0,Yes,Female,Yes
5733,93700,NaN,NaN,NaN,1,1,35,3,2.0,1.0,...,173.3,26.0,40.3,37.5,30.6,98.9,2.0,Yes,Male,No


In [157]:
# Select only people between 20 and 25 years old for RIDAGEYR 
# and store in a new dataframe 'dx'

dx =smoke_da[(smoke_da['RIDAGEYR']>=20) & (smoke_da['RIDAGEYR']<=25)].copy()
dx

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210,SMQ020x,RIAGENDRx,DMDCITZNx
6,83741,1.0,NaN,8.0,1,1,22,4,1.0,4.0,...,165.4,28.0,38.8,38.0,34.0,86.6,NaN,Yes,Male,Yes
17,83761,1.0,NaN,1.0,2,2,24,5,2.0,5.0,...,156.4,25.3,37.0,35.5,29.6,79.5,NaN,No,Female,No
26,83784,1.0,NaN,4.0,1,1,22,2,1.0,4.0,...,170.7,25.3,41.2,36.0,29.7,86.2,2.0,Yes,Male,Yes
38,83809,2.0,2.0,NaN,2,2,20,4,1.0,3.0,...,166.8,26.2,39.2,36.6,31.6,92.6,NaN,No,Female,Yes
40,83813,1.0,NaN,2.0,1,1,24,3,1.0,4.0,...,182.2,26.9,44.2,40.2,33.9,89.2,2.0,Yes,Male,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5688,93618,2.0,2.0,NaN,2,1,25,1,2.0,4.0,...,161.7,27.3,37.4,36.1,31.5,87.3,NaN,No,Male,No
5701,93639,1.0,NaN,2.0,2,1,25,3,1.0,5.0,...,186.1,24.9,48.9,39.8,34.8,88.5,2.0,No,Male,Yes
5707,93653,1.0,NaN,3.0,2,2,25,3,1.0,5.0,...,169.3,26.8,41.8,35.9,29.8,79.0,1.0,No,Female,Yes
5729,93691,2.0,2.0,NaN,2,1,25,5,2.0,5.0,...,136.5,21.0,33.6,29.7,23.8,75.4,2.0,No,Male,No


In [158]:
reset = {'Yes':1,'No':0}
dx = dx.replace({'SMQ020x':reset})
dz=dx[['SMQ020x','RIAGENDRx']]
dz = dz[['SMQ020x','RIAGENDRx']].groupby('RIAGENDRx').agg(['mean','size'])
dz

SMQ020x     
               mean size
RIAGENDRx               
Female     0.238971  272
Male       0.341270  252

Test statistic = $\frac{Best Estimate - Hypothesized Mean}{Standard Error Estimate}$

$=\frac{\hat{p}_1-\hat{p}_2}{se(\hat{p})}$

$se(\hat{p})=\sqrt{\hat{p}(1-\hat{p})(\frac{1}{n_1}+\frac{1}{n_2})}$

where $\hat{p}$ is the population proportion mean for smokers, `SMQ020x == "Yes"`.

In [139]:
# the pooled rate of yes responses, and standard error of the estimated difference of proportions

import statsmodels.stats.proportion as sm_prop
successes = len(dx[dx['SMQ020x'] ==1])
totals = len(dx)
proportion, conf_interval = sm_prop.proportion_confint(successes, totals, method='normal')
print("Proportion of 'yes' responses:", proportion)
se = np.sqrt((proportion * (1 - proportion)) / 524)
print("se",se)

Proportion of 'yes' responses: 0.24938916712275394
se 0.018900816374618525


In [140]:
dx_females = dx.loc[dx.RIAGENDRx == "Female", "SMQ020x"].replace({"Yes": 1, "No": 0})
dx_males = dx.loc[dx.RIAGENDRx == "Male", "SMQ020x"].replace({"Yes": 1, "No": 0})

# Perform the independent t-test
tstat, pvalue, _ = sm.stats.ttest_ind(dx_females, dx_males)

# Print the results without degrees of freedom
print("T-statistic:", tstat)
print("P-value:", pvalue)

T-statistic: -2.5949731446269344
P-value: 0.00972590232121254


Essentially the same test as above can be conducted by converting the "Yes"/"No" responses to numbers (Yes=1, No=0) and conducting a two-sample t-test, as below:

In [125]:
dx_females = dx.loc[dx.RIAGENDRx=="Female", "SMQ020x"].replace({"Yes": 1, "No": 0})
dx_males = dx.loc[dx.RIAGENDRx=="Male", "SMQ020x"].replace({"Yes": 1, "No": 0})
sm.stats.ttest_ind(dx_females, dx_males) # prints test statistic, p-value, degrees of freedom

(-2.5949731446269344, 0.00972590232121254, 522.0)

### Hypothesis tests comparing means

Tests of means are similar in many ways to tests of proportions.  Just as with proportions, for comparing means there are one and two-sample tests, z-tests and t-tests, and one-sided and two-sided tests.  As with tests of proportions, one-sample tests of means are not very common, but we illustrate a one sample test in the cell below.  We compare systolic blood pressure to the fixed value 120 (which is the lower threshold for "pre-hypertension"), and find that the mean is significantly different from 120 (the point estimate of the mean is 126).

In [159]:
# Drop missing values from BPXSY1, RIDAGEYR and RIAGENDRx
bp_da = da.copy()
bp_da.dropna(subset=['BPXSY1','RIDAGEYR','RIAGENDRx'], inplace=True)
bp_da = bp_da[['BPXSY1','RIDAGEYR','RIAGENDRx']]

In [160]:
# Select only "Male" between the ages >= 40 and <= 50
bp_da_filtered = bp_da[(bp_da['RIAGENDRx'] == "Male") & (bp_da['RIDAGEYR']>=40)&(bp_da['RIDAGEYR']<=50)]
bp_da_filtered

,BPXSY1,RIDAGEYR,RIAGENDRx
10,144.0,46,Male
11,116.0,45,Male
20,110.0,49,Male
42,128.0,42,Male
51,118.0,50,Male
...,...,...,...
5680,134.0,50,Male
5690,138.0,48,Male
5693,96.0,41,Male
5713,116.0,43,Male


In [161]:
## calculate the mean blood pressure
bp_da_filtered['BPXSY1'].mean()

125.86698337292161

In [162]:
# using stats.ztest library print the test statistic, p-value where the average is 120
from statsmodels.stats.weightstats import ztest as ztest
ztest(dx2['BPXSY1'], value=120)

(7.469764137102597, 8.033869113167905e-14)

In the cell below, we carry out a formal test of the null hypothesis that the mean blood pressure for women between the ages of 50 and 60 is equal to the mean blood pressure of men between the ages of 50 and 60.  The results indicate that while the mean systolic blood pressure for men is slightly greater than that for women (129 mm/Hg versus 128 mm/Hg), this difference is not statistically significant. 

There are a number of different variants on the two-sample t-test. Two often-encountered variants are the t-test carried out using the t-distribution, and the t-test carried out using the normal approximation to the reference distribution of the test statistic, often called a z-test.  Below we display results from both these testing approaches.  When the sample size is large, the difference between the t-test and z-test is very small.  

In [163]:
# Drop missing values from BPXSY1, RIDAGEYR and RIAGENDRx
all_bp_da = da.copy()
all_bp_da.dropna(subset=['BPXSY1','RIDAGEYR','RIAGENDRx'], inplace=True)
all_bp_da = all_bp_da[['BPXSY1','RIDAGEYR','RIAGENDRx']]

In [164]:
# Select only people from the ages >= 50 and <= 60
all_bp_da_filtered =all_bp_da[(all_bp_da['RIDAGEYR']>=50) & (all_bp_da['RIDAGEYR']<=60)]
all_bp_da_filtered

,BPXSY1,RIDAGEYR,RIAGENDRx
1,146.0,53,Male
3,132.0,56,Female
9,178.0,56,Male
15,134.0,57,Female
19,136.0,54,Female
...,...,...,...
5675,136.0,55,Female
5680,134.0,50,Male
5681,124.0,57,Female
5686,132.0,60,Female


In [165]:
## Select only female
alt_female = all_bp_da_filtered[all_bp_da_filtered['RIAGENDRx'] == 'Female']
alt_female

,BPXSY1,RIDAGEYR,RIAGENDRx
3,132.0,56,Female
15,134.0,57,Female
19,136.0,54,Female
23,116.0,58,Female
27,142.0,60,Female
...,...,...,...
5648,146.0,57,Female
5661,114.0,55,Female
5675,136.0,55,Female
5681,124.0,57,Female


In [166]:
## Select only male
alt_male = all_bp_da_filtered[all_bp_da_filtered['RIAGENDRx'] == 'Male']
alt_male

,BPXSY1,RIDAGEYR,RIAGENDRx
1,146.0,53,Male
9,178.0,56,Male
24,136.0,56,Male
28,132.0,51,Male
32,114.0,56,Male
...,...,...,...
5659,116.0,53,Male
5663,108.0,52,Male
5666,106.0,50,Male
5680,134.0,50,Male


In [167]:
## prints female mean, male mean
print(alt_female['BPXSY1'].mean(),alt_male['BPXSY1'].mean())

127.92561983471074 129.23829787234044


In [168]:
## use the sm.stats.ztest, print the test statistic, p-value
from statsmodels.stats.weightstats import ztest
sample1 = alt_female['BPXSY1']
sample2 = alt_male['BPXSY1']
ztest(sample1,sample2)

(-1.105435895556249, 0.2689707570859362)

In [170]:
## use the sm.stats.ttest_ind, print the test statistic, p-value
import statsmodels.api as sm

sm.stats.ttest_ind(sample1,sample2)

(-1.105435895556249, 0.26925004137768577, 952.0)

Another important aspect of two-sample mean testing is "heteroscedasticity", meaning that the variances within the two groups being compared may be different.  While the goal of the test is to compare the means, the variances play an important role in calibrating the statistics (deciding how big the mean difference needs to be to be declared statistically significant).  In the NHANES data, we see that there are moderate differences between the amount of variation in BMI for females and for males, looking within 10-year age bands.  In every age band, females having greater variation than males.

In [173]:
## Select BMXBMI, RIDAGEYR, RIAGENDRx and drop all NAs
bmi_da = da.copy()
bmi_da.dropna(subset=['BMXBMI','RIDAGEYR','RIAGENDRx'], inplace=True)
bmi_da = bmi_da[['BMXBMI','RIDAGEYR','RIAGENDRx']]
bmi_da.head()

,BMXBMI,RIDAGEYR,RIAGENDRx
0,27.8,62,Male
1,30.8,53,Male
2,28.8,78,Male
3,42.4,56,Female
4,20.3,42,Female


In [175]:
## cut to [18, 30, 40, 50, 60, 70, 80] age group, and calculate the standard deviation
import math
age_groups = [18,30,40,50,60,70,80]
da['agegrp']=pd.cut(da['RIDAGEYR'],bins = age_groups,right=True)
std_dev_female = da[da['RIAGENDRx']=="Female"].groupby('agegrp')['BMXBMI'].std()
std_dev_male = da[da['RIAGENDRx']=="Male"].groupby('agegrp')['BMXBMI'].std()
result = pd.DataFrame({'Female':std_dev_female,'male':std_dev_male})
result

,Female,male
agegrp,,
"(18, 30]",7.745893,6.649440
"(30, 40]",8.315608,6.622412
"(40, 50]",8.076195,6.407076
"(50, 60]",7.575848,5.914373
"(60, 70]",7.604514,5.933307
"(70, 80]",6.284968,4.974855


The standard error of the mean difference (e.g. mean female blood pressure minus mean male blood pressure) can be estimated in at least two different ways.  In the statsmodels library, these approaches are referred to as the "pooled" and the "unequal" approach to estimating the variance.  If the variances are equal (i.e. there is no heteroscedasticity), then there should be little difference between the two approaches.  Even in the presence of moderate heteroscedasticity, as we have here, we can see that the results for the two methods are quite similar.  Below we have a loop that considers each 10-year age band and assesses the evidence for a difference in mean BMI for women and for men.  The results printed in each row of output are the test-statistic and p-value.

In [176]:
for k, v in da.groupby("agegrp"):
    bmi_female = v.loc[v.RIAGENDRx=="Female", "BMXBMI"].dropna()
    bmi_female = sm.stats.DescrStatsW(bmi_female)
    bmi_male = v.loc[v.RIAGENDRx=="Male", "BMXBMI"].dropna()
    bmi_male = sm.stats.DescrStatsW(bmi_male)
    print(k)
    print("pooled: ", sm.stats.CompareMeans(bmi_female, bmi_male).ztest_ind(usevar='pooled'))
    print("unequal:", sm.stats.CompareMeans(bmi_female, bmi_male).ztest_ind(usevar='unequal'))
    print()

(18, 30]
pooled:  (1.702693293364314, 0.08862548061450115)
unequal: (1.7174610823927015, 0.08589495934713479)

(30, 40]
pooled:  (1.4378280405644988, 0.15048285114647975)
unequal: (1.4437869620833568, 0.1487989105789227)

(40, 50]
pooled:  (2.8933761158070186, 0.003811246059501354)
unequal: (2.9678691663536725, 0.0029987194174035366)

(50, 60]
pooled:  (3.3621087799813747, 0.0007734964571391533)
unequal: (3.375494390173931, 0.0007368319423226358)

(60, 70]
pooled:  (3.6172401442432602, 0.0002977610210319532)
unequal: (3.628483094544545, 0.0002850914147149385)

(70, 80]
pooled:  (2.926729252512241, 0.003425469414486057)
unequal: (2.9377798867692064, 0.0033057163315194853)



### Paired tests

A common situation in applied research is to measure the same quantity multiple times on each unit of analysis.  For example, in NHANES, systolic blood pressure is measured at least two times (sometimes there is a third measurement) on each subject.  Although the measurements are repeated, there is no guarantee that the mean is the same each time, i.e. the mean blood pressure may be slightly lower on the second measurement compared to the first, since people are a bit more nervous the first time they are measured.  A paired test is a modified form of mean test that can be used when we are comparing two repeated measurements on the same unit.

A paired t-test for means is equivalent to taking the difference between the first and second measurement, and using a one-sample test to compare the mean of these differences to zero. Below we see that in the entire NHANES sample, the first measurement of systolic blood pressure is on average 0.67 mm/Hg greater than the second measurement.  While this difference is not large, it is strongly statistically significant.  That is, there is strong evidence that the mean values for the first and second blood pressure measurement differ.

In [177]:
## select only BPXSY1 and BPXSY2, drop all NAs
paired_da = da.copy()
paired_da.dropna(subset = ['BPXSY1','BPXSY2'],inplace=True)
paired_da = paired_da[['BPXSY1','BPXSY2']]
paired_da

,BPXSY1,BPXSY2
0,128.0,124.0
1,146.0,140.0
2,138.0,132.0
3,132.0,134.0
4,100.0,114.0
...,...,...
5730,112.0,112.0
5731,118.0,116.0
5732,154.0,146.0
5733,104.0,106.0


In [179]:
##find the difference between BPXSY1 and BPXSY2
paired_da['diff'] = paired_da['BPXSY1'] - paired_da['BPXSY2']
paired_da['diff']

0        4.0
1        6.0
2        6.0
3       -2.0
4      -14.0
        ... 
5730     0.0
5731     2.0
5732     8.0
5733    -2.0
5734     4.0
Name: diff, Length: 5369, dtype: float64

In [180]:
## calculate the mean difference
paired_da['diff'].mean()

0.6749860309182343

In [181]:
## use the sm.stats.ztest library to print test statistic and p-value
from statsmodels.stats.weightstats import ztest
sample = paired_da['diff']
ztest(sample,value=0)

(9.800634425497911, 1.1188070930963587e-22)

To probe this effect further, we can divide the population into 10 year wide age bands and also stratify by gender, then carry out the paired t-test within each of the resulting 12 strata.  We see that the second systolic blood pressure measurement is always lower on average than the first.  The difference is larger for older people and for males.  The difference is statistically significant for females over 30, and for males over 60.   

Conducting many hypothesis tests and "cherry picking" the interesting results is usually a bad practice.  Here we are doing such "multiple testing" for illustration, and acknowledge that the strongest differences may be over-stated.  Nevertheless, there is a clear and consistent trend with age -- older people tend to have greater differences between their first and second blood pressure measurements than younger people.  There is also a difference between the genders, with older men having a stronger difference between the first and second blood pressure measurements than older women.  The gender difference for younger peple is less clear.

In [32]:
dx = da[["RIAGENDRx", "BPXSY1", "BPXSY2", "RIDAGEYR"]].dropna()
dx["agegrp"] = pd.cut(dx.RIDAGEYR, [18, 30, 40, 50, 60, 70, 80])
for k, g in dx.groupby(["RIAGENDRx", "agegrp"]):
    db = g.BPXSY1 - g.BPXSY2
    # print stratum definition, mean difference, sample size, test statistic, p-value
    print(k, db.mean(), db.size, sm.stats.ztest(db.values, value=0))

('Female', Interval(18, 30, closed='right')) 0.13708260105448156 569 (0.7612107360791227, 0.4465312067051751)
('Female', Interval(30, 40, closed='right')) 0.6713615023474179 426 (3.307398751951031, 0.0009416674523368051)
('Female', Interval(40, 50, closed='right')) 0.5970149253731343 469 (2.6040611621024654, 0.009212631487347644)
('Female', Interval(50, 60, closed='right')) 0.7685393258426966 445 (3.1023718750881724, 0.001919766301204196)
('Female', Interval(60, 70, closed='right')) 0.8787878787878788 396 (3.1024528501809625, 0.0019192411825181255)
('Female', Interval(70, 80, closed='right')) 1.4512820512820512 390 (5.141706875154317, 2.722536503552981e-07)
('Male', Interval(18, 30, closed='right')) 0.00390625 512 (0.01959622841647691, 0.9843654725443948)
('Male', Interval(30, 40, closed='right')) 0.46296296296296297 432 (1.9451535788714596, 0.05175649697939119)
('Male', Interval(40, 50, closed='right')) 0.17894736842105263 380 (0.7201800810138878, 0.47141412641258706)
('Male', Interva